In [15]:
# import libraries
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_text_splitters import (
    TokenTextSplitter)

import pandas as pd
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings

import os
import re

In [16]:
# import data
markdown_files = []
for root, dirs, files in os.walk("../data/processed_files"):
    for file in files:
        if file.lower().endswith('.md'):
            markdown_files.append(os.path.join(root, file))
    

In [17]:
# Iterate over the file paths
loaded_documents = []
for doc in markdown_files:
    try:
        loader = UnstructuredMarkdownLoader(doc)
        documents = loader.load()
        loaded_documents.extend(documents)
        print(f"Loaded: {doc}")
    except Exception as e:
        print(f"Error loading {doc}: {str(e)}")

Loaded: ../data/processed_files/Fortportal FY2020/Fortportal Regional Referral Hospital Report of Auditor General 2020.md
Loaded: ../data/processed_files/Fortportal FY2021/Fortportal Regional Referral Hospital Report FY20202021.md
Loaded: ../data/processed_files/Gulu FY2021/Gulu DLG Report of Auditor General 2021.md
Loaded: ../data/processed_files/CAG FY2022/Annual Consolidated OAG audit reports 2022.md
Loaded: ../data/processed_files/MWTS FY2021/MWTS Report of Auditor General 2021.md
Loaded: ../data/processed_files/Fortportal FY2022/Fortportal Regional Referral Hospital Report of Auditor General 2022.md
Loaded: ../data/processed_files/Gulu FY2022/Gulu DLG Report of Auditor General 2022.md
Loaded: ../data/processed_files/CAG FY2021/Annual Consolidated OAG audit reports 2021.md
Loaded: ../data/processed_files/MWTS FY2022/MWTS Report of Auditor General 2022.md


In [18]:
# 2. Custom function to normalize text
def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove special characters (customize as needed)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Apply normalization to each document
for doc in loaded_documents:
    doc.page_content = normalize_text(doc.page_content)

In [19]:
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=150, length_function=len
)
docs = text_splitter.split_documents(loaded_documents)

In [20]:
from dotenv import load_dotenv
load_dotenv()

if "HF_API_KEY" not in os.environ:
    os.environ["HF_API_KEY"] = getpass.getpass("Enter HF API key:")

In [21]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-70B",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

generator_llm = llm
critic_llm = llm
embeddings = embeddings

/Users/jonas/miniconda3/envs/haystack-rag/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {  # uniform distribution
    simple: 0.1,
    reasoning: 0.35,
    multi_context: 0.2,
    conditional: 0.35
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(docs, 10, distributions, raise_exceptions=False) 
testset.to_pandas()

embedding nodes:   0%|          | 0/3810 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# works but is quite expensive with OpenAI API token. 